# Python Speech Recognition and TTS Playback
This little program plugs into Google's speech recognition API to transcribe what you say in a short recording. The default recording time is 5 seconds, but this can be changed in the Parameters section. Once your voice recording has been transcribed, it will be played back to you via text to speech. This is a very simple application of artificial intelligence that can be understood relatively easily.

## Parameters

In [14]:
rec_time = 5

## Dependencies

In [6]:
!apt-get install portaudio19-dev python-all-dev
!apt install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg libav-tools
!pip3 install SpeechRecognition
!pip3 install pyaudio
!pip3 install gTTS
!pip3 install pydub

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-all-dev is already the newest version (2.7.15~rc1-1).
portaudio19-dev is already the newest version (19.6.0-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libasound2-dev' instead of 'libasound-dev'
Package libav-tools is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  ffmpeg

E: Package 'libav-tools' has no installation candidate


## Imports

In [15]:
# all imports
from IPython.display import Javascript, Audio
from google.colab import output
from base64 import b64decode
import speech_recognition as spreg
import os
from pydub import AudioSegment
from gtts import gTTS

## Record Function

In [16]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.webm','wb') as f:
    f.write(b)
  a = AudioSegment.from_file('audio.webm')
  a.export('audio.wav', format='wav')
  return 'audio.webm'  # or webm ?

## Speech Recognition and TTS
Play this as many times as you want to test out the speech recognition and text to speech capabilities of Python.

In [21]:
record(rec_time)
sound_file = 'audio.wav'
recog = spreg.Recognizer()
with spreg.AudioFile(sound_file) as source:
   speech = recog.record(source) #use record instead of listning
   try:
      text = recog.recognize_google(speech)
      audio = gTTS(text=text, lang='en', slow=False)
      audio.save("tts.mp3")
      print(text)
   except spreg.UnknownValueError:
      print('Unable to recognize the audio')
   except spreg.RequestError as e: 
      print("Request error from Google Speech Recognition service; {}".format(e))
Audio(filename="tts.mp3", autoplay=True)

<IPython.core.display.Javascript object>

machine learning is very interesting
